<img src="img/logo.png">

# Chargement des librairies

In [1]:
import numpy as np
import pandas as pd
from mylib.fonction_exploration import exploration
from mylib.fonction_pk import test_pk
import warnings
warnings.filterwarnings('ignore')

print("Version numpy : " + np.__version__)
print("Version pandas : " + pd.__version__)

bold = "\033[1m"
red = "\033[31m"
end = "\033[0;0m"

chemin_erp = 'src/erp.xlsx'
chemin_liaison = 'src/liaison.xlsx'
chemin_web = 'src/web.xlsx'

Version numpy : 1.23.5
Version pandas : 1.4.4


# Chargement des fichiers

In [2]:
erp = pd.read_excel(chemin_erp)
liaison = pd.read_excel(chemin_liaison)
web = pd.read_excel(chemin_web)

# **Erp**

Le fichier ne nécessite pas de nettoyage, il est prêt à l'emploi 

# **Liaison**

## Au vue des futures jointures renommage des colonnes

In [3]:
liaison.rename(columns = {'id_web': 'sku'}, inplace=True)

## Affichage des Nan dans la colonne "sku"

In [4]:
pd.options.display.max_rows = None

In [5]:
nb_nan = liaison['sku'].isna().sum()
print(bold + red + f"\nAu départ, il y a {nb_nan} NaN dans la colonne 'sku' :" + end)
liaison[liaison['sku'].isna()].head() #pour voir la liste complète des 91 doublons supprimer .head()


Au départ, il y a 91 NaN dans la colonne 'sku' :


,product_id,sku
19,4055,NaN
49,4090,NaN
50,4092,NaN
119,4195,NaN
131,4209,NaN


## Pour une meilleure lisibilité suppression des valeurs NaN dans la colonne "sku"

In [6]:
liaison = liaison.dropna()

In [7]:
nb_nan = liaison['sku'].isna().sum()
print(bold + red + f"\nAprès nettoyage, il y a {nb_nan} NaN dans la colonne 'sku' :" + end)
liaison[liaison['sku'].isna()]


Après nettoyage, il y a 0 NaN dans la colonne 'sku' :


,product_id,sku


# **Web**

## Suppression des doublons

In [8]:
duplicates_before = web.duplicated().sum()
print(bold + red + f"\nAu départ, le fichier contient les {duplicates_before} doublons suivants :" + end)
web[web.duplicated()].head() #pour voir la liste complète des 82 doublons supprimer .head()


Au départ, le fichier contient les 82 doublons suivants :


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
179,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
web = web.drop_duplicates()
duplicates_after = web.duplicated().sum()
print(bold + red + f"\nAprès nettoyage, le fichier contient {duplicates_after} doublon :" + end)
web[web.duplicated()]


Après nettoyage, le fichier contient 0 doublon :


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count


## Suppression des colonnes vides

In [10]:
cols_before = len(web.axes[1])
print(bold + red + f"\nAu départ, le fichier contient {cols_before} colonnes dont 4 vides :" + end)
web[['tax_class', 'post_content', 'post_password', 'post_content_filtered']].head() 


Au départ, le fichier contient 28 colonnes dont 4 vides :


,tax_class,post_content,post_password,post_content_filtered
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [11]:
web = web.drop(columns=['tax_class', 'post_content', 'post_password', 'post_content_filtered'])
cols_after = len(web.axes[1])
print(bold + red + f"\nAprès nettoyage, le fichier contient les {cols_after} colonnes suivantes :" + end)
web.columns


Après nettoyage, le fichier contient les 24 colonnes suivantes :


Index(['sku', 'virtual', 'downloadable', 'rating_count', 'average_rating',
       'total_sales', 'tax_status', 'post_author', 'post_date',
       'post_date_gmt', 'post_title', 'post_excerpt', 'post_status',
       'comment_status', 'ping_status', 'post_name', 'post_modified',
       'post_modified_gmt', 'post_parent', 'guid', 'menu_order', 'post_type',
       'post_mime_type', 'comment_count'],
      dtype='object')

## Suppression des lignes en double de la colonne "sku" (restent celles avec le post_type 'product')

In [12]:
duplicates_before = web['sku'].duplicated().sum()
print(bold + red + f"\nAu départ, la colonne 'sku' contient les {duplicates_before} doublons suivants :" + end)
web[web['sku'].duplicated()].head(3) #pour voir la liste complète des 716 doublons supprimer .head(3)


Au départ, la colonne 'sku' contient les 716 doublons suivants :


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,post_date_gmt,...,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
470,NaN,0,0,0,0.0,0.0,taxable,2.0,2018-07-31 12:07:23,2018-07-31 10:07:23,...,closed,pierre-jean-villa-cote-rotie-fongeant-2017,2019-11-02 13:24:15,2019-11-02 12:24:15,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
471,NaN,0,0,0,0.0,0.0,taxable,2.0,2018-08-08 11:23:43,2018-08-08 09:23:43,...,closed,pierre-jean-villa-condrieu-suspendu-2018,2019-11-02 13:24:01,2019-11-02 12:24:01,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
799,15298,0,0,0,0.0,6.0,NaN,2.0,2018-02-08 12:58:52,2018-02-08 11:58:52,...,closed,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


In [13]:
web = web.drop_duplicates(subset=['sku'])
duplicates_after = web['sku'].duplicated().sum()
print(bold + red + f"\nAprès nettoyage, la colonne 'sku' contient {duplicates_after} doublon :" + end)
web[web['sku'].duplicated()]


Après nettoyage, la colonne 'sku' contient 0 doublon :


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,post_date_gmt,...,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,post_mime_type,comment_count


## Suppression de la colonne post_mime_type qui contenait uniquement des valeurs NaN

In [14]:
print(bold + red + f"\nÉléments dans la colonne 'post_mime_type' :" + end)
web['post_mime_type'].unique()


Éléments dans la colonne 'post_mime_type' :


array([nan], dtype=object)

In [15]:
web = web.drop(columns='post_mime_type')

## Suppression de la ligne NaN de la colonne "sku"

In [16]:
print(bold + red + f"\nLes lignes de la colonne 'sku' contenant des NaN sont :" + end)
web[web.isna().any(axis=1)]


Les lignes de la colonne 'sku' contenant des NaN sont :


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,post_date_gmt,...,comment_status,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,comment_count
178,NaN,0,0,0,NaN,NaN,NaN,NaN,NaT,NaT,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [17]:
web = web.drop(index=178)
web_nan = web.isna().sum()
print(bold + red + f"\nAprès nettoyage, le nombre de NaN par colonne est de :" + end)
web_nan


Après nettoyage, le nombre de NaN par colonne est de :


sku                  0
virtual              0
downloadable         0
rating_count         0
average_rating       0
total_sales          0
tax_status           0
post_author          0
post_date            0
post_date_gmt        0
post_title           0
post_excerpt         0
post_status          0
comment_status       0
ping_status          0
post_name            0
post_modified        0
post_modified_gmt    0
post_parent          0
guid                 0
menu_order           0
post_type            0
comment_count        0
dtype: int64

# **Jointure** entre liaison et web

In [18]:
# vérification de l'unicité de la clé candidate
test_pk(liaison, 'sku')

La clé est unique


In [19]:
# vérification de l'unicité de la clé candidate
test_pk(web, 'sku')

La clé est unique


In [20]:
# jointure avec indicateur
liaison_web = liaison.merge(web, on='sku', how='outer', indicator=True)
liaison_web_left = liaison_web[liaison_web['_merge'] == 'left_only']
web_nan = web.isna().sum()
print(bold + red + f"\n{len(liaison_web_left)} références produits de la table LIAISON n'ont pas de description dans la table WEB :" + end)
liaison_web_left


20 références produits de la table LIAISON n'ont pas de description dans la table WEB :


,product_id,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,...,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,comment_count,_merge
185,4289,13771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
227,4568,15065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
230,4584,14785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
334,4741,12601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
368,4864,15154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
371,4869,14360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
399,4921,15608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
400,4922,15586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
443,5018,15272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only
445,5021,15630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,left_only


In [32]:
# choix de la jointure inner pour la suite de l'analyse
liaison_web = liaison_web[liaison_web['_merge'] == 'both']
liaison_web.sample(3)

,product_id,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,post_author,post_date,...,ping_status,post_name,post_modified,post_modified_gmt,post_parent,guid,menu_order,post_type,comment_count,_merge
391,4910,13809,0.0,0.0,0.0,0.0,4.0,taxable,2.0,2018-05-15 11:11:59,...,closed,antoine-marie-arena-vin-de-france-rouge-san-gi...,2020-04-23 22:54:55,2020-04-23 20:54:55,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,0.0,both
706,6621,15204,0.0,0.0,0.0,0.0,3.0,taxable,2.0,2020-01-18 10:54:27,...,closed,thierry-germain-saumur-champigny-la-marginale-...,2020-06-18 10:45:04,2020-06-18 08:45:04,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,0.0,both
55,4101,14905,0.0,0.0,0.0,0.0,0.0,taxable,2.0,2018-02-13 10:11:16,...,closed,emile-boeckel-pinot-gris-gc-zotzenberg-2016,2020-08-25 18:45:02,2020-08-25 16:45:02,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,0.0,both


# **Jointure** entre erp et liaison

In [22]:
# vérification de l'unicité de la clé candidate
test_pk(erp, 'product_id')

La clé est unique


In [23]:
# vérification de l'unicité de la clé candidate
test_pk(liaison, 'product_id')

La clé est unique


In [24]:
# jointure
erp_liaison = erp.merge(liaison, on='product_id', how='outer', indicator=True)
erp_liaison = erp_liaison[erp_liaison['_merge'] == 'both']
erp_liaison.sample(5)

,product_id,onsale_web,price,stock_quantity,stock_status,sku,_merge
758,6738,1,15.4,40,instock,8193,both
370,4782,1,9.8,22,instock,16564,both
453,4977,1,16.3,47,instock,14930,both
713,6221,1,23.5,3,instock,16096,both
8,4043,1,60.0,12,instock,14980,both


# **Jointure** entre les deux tables précédemment créées

In [25]:
# vérification de l'unicité de la clé candidate
test_pk(erp_liaison, 'sku')

La clé est unique


In [26]:
# vérification de l'unicité de la clé candidate
test_pk(liaison_web, 'sku')

La clé est unique


In [27]:
# jointure
erp_web = erp_liaison.merge(liaison_web, on='sku', how='inner')

In [28]:
# remodelage du DataFrame
erp_web = erp_web.drop(columns=['_merge_x', 'product_id_y', 'virtual',
                                'downloadable', '_merge_y', 'menu_order', 
                                'post_parent', 'comment_count'])
erp_web.rename(columns={'product_id_x':'product_id'}, inplace=True)
erp_web.sample(3)

,product_id,onsale_web,price,stock_quantity,stock_status,sku,rating_count,average_rating,total_sales,tax_status,...,post_title,post_excerpt,post_status,comment_status,ping_status,post_name,post_modified,post_modified_gmt,guid,post_type
323,4728,1,29.5,20,instock,15729,0.0,0.0,2.0,taxable,...,Maurice Schoech Gewurztraminer Vendanges Tardi...,La peau épaisse et rosée du gewurztraminer se ...,publish,closed,closed,schoech-gewurztraminer-vt-2017,2020-08-25 18:05:02,2020-08-25 16:05:02,https://www.bottle-neck.fr/?post_type=product&...,product
690,6627,1,41.8,16,instock,15791,0.0,0.0,1.0,taxable,...,Weingut Besson-Strasser Zürich Fumé 2017,Assemblage équilibré entre Chardonnay et Räusc...,publish,closed,closed,weingut-besson-strasser-zurich-fume-2017,2020-08-11 11:25:02,2020-08-11 09:25:02,https://www.bottle-neck.fr/?post_type=product&...,product
337,4757,1,26.5,1,instock,14680,0.0,0.0,6.0,taxable,...,Domaine Saint-Nicolas Fiefs Vendéens Blanc Le ...,"Le Haut des Clous déploie une matière robuste,...",publish,closed,closed,domaine-saint-nicolas-fiefs-vendeens-blanc-le-...,2020-08-08 17:45:03,2020-08-08 15:45:03,https://www.bottle-neck.fr/?post_type=product&...,product


In [29]:
print(erp_web.shape)

(714, 23)


<span style="color:red">**=> Après nettoyage et jointure des 3 tables, 714 références sont exploitables pour l'analyse**</span>

# **Conclusion** sur les fichiers à l'attention des stakeholders

## ***Résumé des tâches effectuées***

+ **ERP** => RAS, fichier prêt pour l'analyse


+ **LIAISON** => 
            
            1. renommage de la colonne "id_web" en "sku"
            2. suppression des NaN dans la colonne "sku"
            
+ **WEB** => 
        
            1. suppression des 82 doublons
            2. suppression des colonnes vides "tax_class", "post_content", "post_password", "post_content_filtered"
            3. suppression des lignes en double de la colonne "sku" (restent celles avec le post_type 'product')
            4. suppression de la colonne post_mime_type qui contenait uniquement des valeurs NaN
            5. suppression de la ligne NaN de la colonne "sku"

## ***Données manquantes***

In [30]:
print(f" Nombres de références produits dans la table ERP : {len(erp)}")
print(f" Nombres de références produits dans la table LIAISON : {len(liaison)}")
print(f" Nombres de références produits dans la table WEB : {len(web)}")

 Nombres de références produits dans la table ERP : 825
 Nombres de références produits dans la table LIAISON : 734
 Nombres de références produits dans la table WEB : 714


Il manque des références et données produits, qu'il conviendrait de compléter pour affiner l'analyse : 

+ Fichier ERP : RAS, 825 références produits
+ Fichier LIAISON : 91 NaN dans la colonne "sku", 734 références produits -> 91 correspondances de références manquantes à compléter
+ Fichier WEB : 4 colonnes vides, 1 NaN dans colonne "sku" (index 178), 20 références produits dans LIAISON sans description dans WEB

# Exportation de la jointure des trois tables vers un fichier CSV

In [31]:
erp_web.to_csv('/Users/zqldata/Documents/P5/Input/erp_web.csv', index=False) 